# Percpetron

In [1]:
import sys

%matplotlib inline
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from glob import glob
import numpy as np

from torchvision import transforms

from plot_mel import * 


from sklearn.utils import gen_batches

from tqdm import tqdm

## Load dataset



In [9]:
encoded_space = torch.load("../Dataset/Encodings/encoded_space_SPA.pt")

### Train test split

In [ ]:
#Since I have problems with the labels, I am not using Dataloader from PyTorch

In [10]:
#You need to remember how much spanish vs non-spanish is there in your encoded space...
labels = [1 for _ in range(1000)] + [0 for _ in range(2000)]

In [11]:
X = encoded_space
y = labels


X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2,random_state=420,shuffle=True)

In [12]:
class Perceptron(nn.Module):
    
    def __init__(self):
        super(Perceptron,self).__init__()
        
        #do I need a Sequential stuff?
        self.model = nn.Sequential(
           
        nn.Linear(in_features=3000, out_features= 1, bias= 0),
        nn.Sigmoid()
            
        )
        
    def forward(self,x):
        x = self.model(x)
        return x
    
perceptron = Perceptron()

#print architecture
print(perceptron)

Perceptron(
  (model): Sequential(
    (0): Linear(in_features=3000, out_features=1, bias=False)
    (1): Sigmoid()
  )
)


In [ ]:
perceptron.load_state_dict(torch.load("../saved_models/perceptronSPA_ENG.pt"))

## Train it

In [13]:
num_epochs = 500
batch_size = 100
learning_rate = 1e-3

In [14]:
X_train.shape

torch.Size([2400, 3000])

In [ ]:
# set to training mode
perceptron.train()


optimizer = torch.optim.Adam(params=perceptron.parameters(), lr=learning_rate, weight_decay=1e-3)

train_loss_avg = []



# Calculate the number of batches
num_batches = len(X_train) // batch_size


print('Training ...')
for epoch in range(num_epochs):
    train_loss_avg.append(0)

    # Generate batches
    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = (batch_idx + 1) * batch_size


        X_batch = X_train[start_idx:end_idx]
        y_batch = y_train[start_idx:end_idx]


        # predict label
        #can simplfy code with no redundancy
        batch_labels = perceptron(X_batch.float().squeeze(1))
        #pytorch wants a float instead of a dtype...
        true_labels = y_batch.float()

        # prediction error
        #label vs predizione

        loss = F.mse_loss(batch_labels.squeeze(1), true_labels)

        # backpropagation
        optimizer.zero_grad()
        loss.backward()

        # one step of the optmizer (using the gradients from backpropagation)
        optimizer.step()

        train_loss_avg[-1] += loss.item()

    train_loss_avg[-1] /= num_batches
    print('Epoch [%d / %d] average prediction error: %f' % (epoch+1, num_epochs, train_loss_avg[-1]))


### Plot training curve

In [ ]:
import matplotlib.pyplot as plt
plt.ion()

fig = plt.figure()
plt.plot(train_loss_avg)
plt.xlabel('Epochs')
plt.ylabel('Prediction error')
plt.title("Perceptron")
plt.show()

## Evaluation


In [ ]:
# set to evaluation mode
perceptron.eval()

# Calculate the number of batches
num_batches = len(X_test) // batch_size

test_loss_avg = 0
for batch_idx in range(num_batches):
    
    start_idx = batch_idx * batch_size
    end_idx = (batch_idx + 1) * batch_size
    
    
    X_batch = X_test[start_idx:end_idx]
    y_batch = y_test[start_idx:end_idx]

    
    
    with torch.no_grad():
        
        # autoencoder reconstruction
        y_batch_prediction = perceptron(X_batch).squeeze(1)
  
        # reconstruction error
        loss = F.mse_loss(y_batch_prediction, y_batch)

        test_loss_avg += loss.item()
        
        
test_loss_avg /= num_batches
print('average prediction error: %f' % (test_loss_avg))

In [ ]:
torch.save(perceptron.state_dict(),"../saved_models/perceptronSPA_ENG_JAP.pt")